<a href="https://colab.research.google.com/github/brit228/DS-Unit-1-Sprint-4-Statistical-Tests-and-Experiments/blob/master/DS_Unit_1_Sprint_Challenge_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Science Unit 1 Sprint Challenge 4

## Exploring Data, Testing Hypotheses

In this sprint challenge you will look at a dataset of people being approved or rejected for credit.

https://archive.ics.uci.edu/ml/datasets/Credit+Approval

Data Set Information: This file concerns credit card applications. All attribute names and values have been changed to meaningless symbols to protect confidentiality of the data. This dataset is interesting because there is a good mix of attributes -- continuous, nominal with small numbers of values, and nominal with larger numbers of values. There are also a few missing values.

Attribute Information:
- A1: b, a.
- A2: continuous.
- A3: continuous.
- A4: u, y, l, t.
- A5: g, p, gg.
- A6: c, d, cc, i, j, k, m, r, q, w, x, e, aa, ff.
- A7: v, h, bb, j, n, z, dd, ff, o.
- A8: continuous.
- A9: t, f.
- A10: t, f.
- A11: continuous.
- A12: t, f.
- A13: g, p, s.
- A14: continuous.
- A15: continuous.
- A16: +,- (class attribute)

Yes, most of that doesn't mean anything. A16 (the class attribute) is the most interesting, as it separates the 307 approved cases from the 383 rejected cases. The remaining variables have been obfuscated for privacy - a challenge you may have to deal with in your data science career.

Sprint challenges are evaluated based on satisfactory completion of each part. It is suggested you work through it in order, getting each aspect reasonably working, before trying to deeply explore, iterate, or refine any given step. Once you get to the end, if you want to go back and improve things, go for it!

## Part 1 - Load and validate the data

- Load the data as a `pandas` data frame.
- Validate that it has the appropriate number of observations (you can check the raw file, and also read the dataset description from UCI).
- UCI says there should be missing data - check, and if necessary change the data so pandas recognizes it as na
- Make sure that the loaded features are of the types described above (continuous values should be treated as float), and correct as necessary

This is review, but skills that you'll use at the start of any data exploration. Further, you may have to do some investigation to figure out which file to load from - that is part of the puzzle.

In [101]:
!wc -l crx.data

690 crx.data


In [103]:
import pandas as pd

df = pd.read_csv("https://archive.ics.uci.edu/ml/machine-learning-databases/credit-screening/crx.data", header=None)
df.columns = ["A{}".format(i) for i in range(1, 17)]

print(df.count())
print()

for a in ["A9", "A10", "A12"]:
  df[a] = df[a].map({"t": True, "f": False})
df["A16"] = df["A16"].map({"+": True, "-": False})
df["A1"] = df["A1"].apply(lambda x: "?" if x not in ["a", "b"] else x)
df["A2"] = df["A2"].apply(lambda x: None if x == "?" else float(x))
df["A4"] = df["A4"].apply(lambda x: "?" if x not in ["u", "y", "l", "t"] else x)
df["A5"] = df["A5"].apply(lambda x: "?" if x not in ["g", "p", "gg"] else x)
df["A6"] = df["A6"].apply(lambda x: "?" if x not in ["c", "d", "cc", "i", "j", "k", "m", "r", "q", "w", "x", "e", "aa", "ff"] else x)
df["A7"] = df["A7"].apply(lambda x: "?" if x not in ["v", "h", "bb", "j", "n", "z", "dd", "ff", "o"] else x)
df["A14"] = df["A14"].apply(lambda x: None if x == "?" else float(x))

for a in df:
  print(a, df[a].dtype)
print()
print("NA Values")
df.isna().sum()

A1     690
A2     690
A3     690
A4     690
A5     690
A6     690
A7     690
A8     690
A9     690
A10    690
A11    690
A12    690
A13    690
A14    690
A15    690
A16    690
dtype: int64

A1 object
A2 float64
A3 float64
A4 object
A5 object
A6 object
A7 object
A8 float64
A9 bool
A10 bool
A11 int64
A12 bool
A13 object
A14 float64
A15 int64
A16 bool

NA Values


A1      0
A2     12
A3      0
A4      0
A5      0
A6      0
A7      0
A8      0
A9      0
A10     0
A11     0
A12     0
A13     0
A14    13
A15     0
A16     0
dtype: int64

## Part 2 - Exploring data, Testing hypotheses

The only thing we really know about this data is that A16 is the class label. Besides that, we have 6 continuous (float) features and 9 categorical features.

Explore the data: you can use whatever approach (tables, utility functions, visualizations) to get an impression of the distributions and relationships of the variables. In general, your goal is to understand how the features are different when grouped by the two class labels (`+` and `-`).

For the 6 continuous features, how are they different when split between the two class labels? Choose two features to run t-tests (again split by class label) - specifically, select one feature that is *extremely* different between the classes, and another feature that is notably less different (though perhaps still "statistically significantly" different). You may have to explore more than two features to do this.

For the categorical features, explore by creating "cross tabs" between them and the class label, and apply the Chi-squared test to them. There are 9 categorical features - as with the t-test, try to find one where the Chi-squared test returns an extreme result (rejecting the null that the data are independent), and one where it is less extreme.

**NOTE** - "less extreme" just means smaller test statistic/larger p-value. Even the least extreme differences may be strongly statistically significant.

Your *main* goal is the hypothesis tests, so don't spend too much time on the exploration/visualization piece. That is just a means to an end. This is challenging, so manage your time and aim for a baseline of at least running two t-tests and two Chi-squared tests before polishing. And don't forget to answer the questions in part 3, even if your results in this part aren't what you want them to be.

In [128]:
from scipy import stats


print("Column\t\t- Mean\t- SEM\t\t+ Mean\t+ SEM\t\tP (ttest)\tDifferent")
a16_df = list(df.groupby("A16"))
for a in [2, 3, 8, 11, 14, 15]:
  vals = []
  for a16 in a16_df:
    vals.append(list(a16[1]["A{}".format(a)].dropna().values))
  pval = stats.ttest_ind(vals[0], vals[1], equal_var=False).pvalue
  v0 = stats.describe(vals[0])
  v1 = stats.describe(vals[1])
  print("A{}\t\t{:.2f}\t{:.2f}\t\t{:.2f}\t{:.2f}\t\t{:.4f}\t\t{}".format(a, v0.mean, stats.sem(vals[0]), v1.mean, stats.sem(vals[1]), pval, pval < 0.05))
  
print()  
print()

print("Column\t\tP (ChiSquare)")
for a in [1,4,5,6,7,9,10,12,13]:
  vals = []
  nam = ""
  ratio = df[df["A16"] == True]["A16"].count() / df["A16"].count()
  for v in sorted(set(df["A{}".format(a)])):
      nam += "\t{}".format(v)
  for a16 in a16_df:
    val = []
    for v in sorted(set(df["A{}".format(a)])):
      val.append(a16[1][a16[1]["A{}".format(a)] == v]["A{}".format(a)].count())
    vals.append(val)
  print()
  print("A{}\t".format(a) + nam)
  chi = stats.chisquare(vals, f_exp=[[ratio * df[df["A{}".format(a)] == v]["A{}".format(a)].count() for v in sorted(set(df["A{}".format(a)]))], [(1 - ratio) * df[df["A{}".format(a)] == v]["A{}".format(a)].count() for v in sorted(set(df["A{}".format(a)]))]])
  s = ""
  for i in chi[1]:
    s += "\t{:.4f}".format(i)
  print("\t" + s)
  
print()  
print()

for a in [1,4,5,6,7,9,10,12,13]:
  print(pd.crosstab(df["A16"], df["A{}".format(a)], dropna=False, margins=True))
  print()
  print()
  print()

Column		- Mean	- SEM		+ Mean	+ SEM		P (ttest)	Different
A2		29.81	0.57		33.72	0.73		0.0000		True
A3		3.84	0.22		5.90	0.31		0.0000		True
A8		1.26	0.11		3.43	0.24		0.0000		True
A11		0.63	0.10		4.61	0.36		0.0000		True
A14		199.70	9.36		164.42	9.32		0.0078		True
A15		198.61	34.32		2038.86	437.17		0.0000		True


Column		P (ChiSquare)

A1		?	a	b
		0.0335	0.0099	0.0000

A4		?	l	u	y
		0.5823	0.2055	0.0046	0.0000

A5		?	g	gg	p
		0.5823	0.0046	0.2055	0.0000

A6		?	aa	c	cc	d	e	ff	i	j	k	m	q	r	w	x
		0.5042	0.0027	0.0158	0.0498	0.0004	0.9605	0.0000	0.0000	0.1046	0.0001	0.0964	0.0792	0.6973	0.5254	0.0004

A7		?	bb	dd	ff	h	j	n	o	v	z
		0.5042	0.0423	0.2744	0.0000	0.0748	0.3054	0.8246	0.8755	0.0000	0.2672

A9		False	True
		0.0000	0.0000

A10		False	True
		0.0000	0.0000

A12		False	True
		0.0000	0.0009

A13		g	p	s
		0.0000	0.6906	0.0000


A1      ?    a    b  All
A16                     
False   9  112  262  383
True    3   98  206  307
All    12  210  468  690



A4     ?  l    u    y  All
A16          

## Part 3 - Analysis and Interpretation

Now that you've looked at the data, answer the following questions:

- Interpret and explain the two t-tests you ran - what do they tell you about the relationships between the continuous features you selected and the class labels?
- Interpret and explain the two Chi-squared tests you ran - what do they tell you about the relationships between the categorical features you selected and the class labels?
- What was the most challenging part of this sprint challenge?

Answer with text, but feel free to intersperse example code/results or refer to it from earlier.

For all continuous variables, the p-values using Welch's t test are all less than 0.01, meaning it is highly likely that the distributions of each class is independent when looking at individual continuous variables. This shows that for all continuous variables, there is not much overlap between the distributions for the two class variables, meaning that they could be good predictors for class attribute.

For the categorical variables, the comparisons need to be done on a case by case basis. However, what the chi-squared p values show is how each category in the variable is representative of the whole distribution of the class attributes. The majority of the p-values fall under 0.01, meaning rejection of the null hypothesis that the category is representative of the categorical variable distribution. The following categories for each categorical variable are those where the null hypothesis is accepted:

- A1: ?
- A4: ?, l
- A5: ?, gg
- A6: ?, c, cc, e, j, m, q, r, w
- A7: ?, bb, dd, h, j, n, o, z
- A13: p

As shown above in the printed crosstab table, the probable reason due to the higher p-values could be that theren just isn't enough data for each category. However for A6(c,cc,e,m,q,w), and A7(bb,h), these categories have counts of over 20, and we can therefore determine that these values are not representative of the class attribute ratio.

The most challenging part of the sprint challenge was trying to understand more about the scipy stats chisquare function and using it properly.